This Notebook takes the combined datasets output from the "CombineDataSets" notebook and performs economic impact analysis base don BPS fines

In [66]:

import re
import os
import glob
import pandas as pd
import numpy as np

In [67]:
#load the Data Sets

property_filepath = r"..\Data Analysis\DataSets\PropertyDataCombinedFinal.csv"
tenant_filepath = r"..\Data Analysis\DataSets\TenantDataCombined.csv"
comstock_metadata_filepath = r"..\Data Analysis\DataSets\ComStockMetaData\comstockmetadataaurora.csv"
average_market_data_filepath = r"DataSets\Raw CoStar Market Data\Average Market Data\Average Market Data Filtered.xlsx"
financial_data_filepath = r"..\Data Analysis\DataSets\Financial data to Merge\financial_data_2022.csv"

property_df = pd.read_csv(property_filepath)
tenant_df = pd.read_csv(tenant_filepath)
comstock_metadata_df = pd.read_csv(comstock_metadata_filepath)
financial_df = pd.read_csv(financial_data_filepath)

average_market_df = pd.read_excel(average_market_data_filepath, sheet_name='Average Market Data')
average_market_hospitality_df = pd.read_excel(average_market_data_filepath, sheet_name='Average Market Data Hospitality')


The Team gathered financial data from CoStar for all the properties. Merge into the Property Data Frame and Normalize if need be

In [68]:
#Map Financial Data to Property DF
financial_df.rename(columns = {'Property ID': 'PropertyID'}, inplace = True)
property_df = pd.merge(property_df,financial_df, how = 'left', on = 'PropertyID')

property_df
#if financial data is over $100 then normalize to $/SF
# Column contains values with $ signs
property_df['Total Operating Expenses'] = pd.to_numeric(property_df['Total Operating Expenses'].replace('[\$,]', '', regex=True), errors='coerce')
property_df['Utilities'] = pd.to_numeric(property_df['Utilities'].replace('[\$,]', '', regex=True), errors='coerce')

# Assuming 'Square foot' is the column representing square footage in your DataFrame
condition = property_df['Total Operating Expenses'] > 100
condition_2 = property_df['Utilities'] > 100
#Calculate Total Area
property_df['Total Area SF'] = property_df['Typical Floor Size'] * property_df['Number Of Stories']

# Update values based on the condition
property_df.loc[condition, 'Total Operating Expenses'] /= property_df.loc[condition, 'Total Area SF']
property_df.loc[condition, 'Utilities'] /= property_df.loc[condition, 'Total Area SF']




First Perform the Analysis on a Property Level. The First part of the Analysis is to take all the locations that dont have EUI data from Touchstone IQ and map the Comstock EUIs to those to a Property based on CoStar Data

In [69]:
#Create a dictioinary for CoStar Building Type to Cmostock Building Type (CoStar Building Type: ComStock Building Type)
costar_to_comstock_builting_type_dict = {
    'Retail: Bar' : 'Full Service Restaurant','Retail: Restaurant' : 'Full Service Restaurant','Health Care: Hospital' : 'Hospital','Hospitality: Hotel' : 'Large Hotel','Hospitality: Hotel casino' : 'Large Hotel','Office' : 'Office','Office: Industrial Live/Work Unit' : 'Office','Office: Office live/work unit' : 'Office','Office: Office/Residential' : 'Office','Retail: Bank' : 'Office','Flex' : 'Office','Flex: R&D' : 'Office','Office: Service' : 'Office','Health Care: Assisted Living' : 'Outpatient','Health Care: Rehabilitation Center' : 'Outpatient','Health Care: Skilled Nursing Facility' : 'Outpatient','Health Care: Congregate Senior Housing' : 'Outpatient','Health Care: Continuing Care Retirement Community' : 'Outpatient','Office: Medical' : 'Outpatient','Health Care' : 'Outpatient','Not applicable' : 'Primary/Secondary School','Not applicable' : 'Primary/Secondary School','Retail: Fast Food' : 'Quick Service Restaurant','General Retail: Fast Food' : 'Quick Service Restaurant','Retail: Department Store' : 'Retail','Retail: Freestanding' : 'Retail','Retail: Garden Center' : 'Retail','General Retail: Freestanding' : 'Retail','Hospitality: Motel' : 'Small hotel','Hospitality' : 'Small hotel','Flex: Showroom' : 'Strip Mall','Retail: Storefront' : 'Strip Mall','Retail: Storefront Retail/Office' : 'Strip Mall','Retail: Storefront retail/residential' : 'Strip Mall','Specialty: Post Office' : 'Strip Mall','Retail' : 'Strip Mall','Retail: Storefront' : 'Strip Mall','Retail: Storefront Retail/Office' : 'Strip Mall','Retail: Freestanding' : 'Strip Mall','Retail: Storefront Retail/Residential' : 'Strip Mall','Retail: Auto Repair' : 'Strip Mall','Retail: Auto Dealership' : 'Strip Mall','Retail: Restaurant' : 'Strip Mall','Retail: Movie Theatre' : 'Strip Mall','Retail: Bank' : 'Strip Mall','Retail: Convenience Store' : 'Strip Mall','Retail: Health Club' : 'Strip Mall','Retail: Day Care Center' : 'Strip Mall','Retail: Supermarket' : 'Strip Mall','Retail: Veterinarian/Kennel' : 'Strip Mall','Retail: Fast Food' : 'Strip Mall','Retail: Drug Store' : 'Strip Mall','Retail: Garden Center' : 'Strip Mall','Retail: Service Station' : 'Strip Mall','Retail: Department Store' : 'Strip Mall','Retail: Retail Building' : 'Strip Mall','Retail: Bar/Nightclub' : 'Strip Mall','Retail: Funeral Home' : 'Strip Mall','Retail: Bowling Alley' : 'Strip Mall','Retail: Quick Service' : 'Strip Mall','General Retail' : 'Strip Mall','Flex: Light Distribution' : 'Warehouse','Flex: Light Manufacturing' : 'Warehouse','Industrial: Distribution' : 'Warehouse','Industrial: Service' : 'Warehouse','Industrial: Showroom' : 'Warehouse','Industrial: Truck Terminal' : 'Warehouse','Industrial: Warehouse' : 'Warehouse','Specialty: Airplane Hangar' : 'Warehouse','Specialty: Self-Storage' : 'Warehouse','Multi-Family' : 'Multi-Family', 'Multi-Family: Apartments':'Multi-Family'}

#Create a dictionary for Colorado BPS 2026 EUI Target (ComStock Building Type: EUI Target)
colorado_bps_2026_eui_targets_dict = {
'Full Service Restaurant' : 120.1, 'Full Service Restaurant' : 269.4, 'Hospital' : 224.5, 'Large Hotel' : 69.1, 'Large Hotel' : 77.1, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Office' : 61.4, 'Outpatient' : 73.9, 'Outpatient' : 73.9, 'Outpatient' : 73.9, 'Outpatient' : 73.9, 'Outpatient' : 73.9, 'Outpatient' : 87.2, 'Outpatient' : 73.9, 'Primary/Secondary School' : 60.7, 'Primary/Secondary School' : 60.7, 'Quick Service Restaurant' : 432, 'Quick Service Restaurant' : 432, 'Retail' : 66.6, 'Retail' : 66.6, 'Retail' : 66.6, 'Retail' : 66.6, 'Small hotel' : 69.1, 'Small hotel' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 64.5, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Strip Mall' : 69.6, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Warehouse' : 42.1, 'Multi-Family' : 55
}

#Create a dictionary for Colorado BPS 2030 EUI Targets (ComStock Building Type: EUI Target)
colorado_bps_2030_eui_targets_dict = {
    'Full Service Restaurant' : 98.8, 'Full Service Restaurant' : 221.5, 'Hospital' : 191.3, 'Large Hotel' : 58.8, 'Large Hotel' : 71, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Office' : 52.4, 'Outpatient' : 60.8, 'Outpatient' : 60.8, 'Outpatient' : 60.8, 'Outpatient' : 60.8, 'Outpatient' : 60.8, 'Outpatient' : 70.2, 'Outpatient' : 60.8, 'Primary/Secondary School' : 53.3, 'Primary/Secondary School' : 53.3, 'Quick Service Restaurant' : 355.2, 'Quick Service Restaurant' : 355.2, 'Retail' : 50, 'Retail' : 50, 'Retail' : 50, 'Retail' : 50, 'Small hotel' : 58.8, 'Small hotel' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 53, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Strip Mall' : 48.9, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Warehouse' : 32.3, 'Multi-Family' : 46.8
}

#Create functions for CoStar to Comstock MetaData Mapping
def costar_to_comstock_vintage(year):
    if year < 1946:
        return 'Before 1946'
    elif year >= 1946 and year < 1960:
        return "1946 to 1959"
    elif year >= 1960 and year < 1970:
        return "1960 to 1969"
    elif year >= 1970 and year < 1980:
        return "1970 to 1979"
    elif year >= 1980 and year < 1990:
        return "1980 to 1989"
    elif year >= 1990 and year < 2000:
        return "1990 to 1999"
    elif year >= 2000 and year < 2013:
        return "2000 to 2012"
    else:
        return "2013 to 2018"


def costar_to_comstock_construction(construction):
    if construction == 'Masonry':
        return "Mass"
    elif construction == 'Reinforced Concrete':
        return 'Mass'
    elif construction == 'Wood Frame':
        return 'WoodFramed'
    elif construction == 'Steel':
        return 'SteelFramed'
    elif construction == 'Metal':
        return 'Metal Building'
    else:
        return 'Wood Frame'


def costar_to_comstock_floor_area(area):
    if area < 1001:
        return 1000
    elif area > 1000 and area < 5001:
        return "1001-5000"
    elif area > 5000 and area < 10001:
        return "5001-10000"
    elif area > 10000 and area < 25001:
        return "10001-25000"
    elif area > 25000 and area < 50001:
        return "25001-50000"
    elif area > 50000 and area < 100001:
        return "50001-100000"
    elif area > 100000 and area < 200001:
        return "100001-200000"
    else:
        return '200001-500000'

    

Utilize the EUIs from TouchStone IQ or utilize ComStock models to assume EUI. If specific ComStock EUI is not available the next closest model will be utilized.

In [70]:
#filter down to the comstock metadata columns needed rename the columns and change the EUI units
selected_columns = ['in.comstock_building_type','in.vintage','in.wall_construction_type','in.floor_area_category','in.number_of_stories','out.site_energy.total.energy_consumption_intensity']
renamed_columns_dict = {'in.comstock_building_type': 'ComStock Property Type','in.vintage': 'ComStock Vintage','in.wall_construction_type': 'ComStock Construction','in.floor_area_category': 'ComStock Size Range','in.number_of_stories': 'Number Of Stories','out.site_energy.total.energy_consumption_intensity': 'EUI (kW/SF)'}
filtered_comstock_metadata_df = comstock_metadata_df[selected_columns]
filtered_comstock_metadata_df = filtered_comstock_metadata_df.rename(columns = renamed_columns_dict)
filtered_comstock_metadata_df['Comstock EUI (BTU/SF)'] = filtered_comstock_metadata_df['EUI (kW/SF)']*3.412
filtered_comstock_metadata_df.drop(columns = 'EUI (kW/SF)', inplace = True)


#clean up the year and floor area data to be a float and if Nan then make it == 0
property_df['Year Built'] = property_df['Year Built'].fillna(0).astype(int)
property_df['Total Area SF'] = property_df['Total Area SF'].fillna(0).astype(int)

#Create a full property type column
property_df['Full Property Type'] = property_df['PropertyType'] + ": " + property_df['Secondary Type']
property_df['Full Property Type'] = property_df['Full Property Type'].fillna('Flex')

#apply the functions to convert costar data to comstock
property_df['ComStock Construction'] = property_df['Construction Material'].apply(costar_to_comstock_construction)
property_df['ComStock Vintage'] = property_df['Year Built'].apply(costar_to_comstock_vintage)
property_df['ComStock Size Range'] = property_df['Total Area SF'].apply(costar_to_comstock_floor_area)


#Map ComStock and BPS EUI Targets to Data set
property_df['ComStock Property Type'] = property_df['Full Property Type'].map(costar_to_comstock_builting_type_dict)
property_df['Colorado 2026 EUI Target'] = property_df['ComStock Property Type'].map(colorado_bps_2026_eui_targets_dict) 
property_df['Colorado 2030 EUI Target'] = property_df['ComStock Property Type'].map(colorado_bps_2030_eui_targets_dict)


#create a seperate dataframe from to CoStar for Comstock Parameters to ensure that Comstock df covers all perumtations
costar_comstock_parameters_df = property_df[['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories']]
filtered_comstock_metadata_df = pd.merge(
    costar_comstock_parameters_df,
    filtered_comstock_metadata_df,
    on = ['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories'], 
    how = 'left'
    )



# average the comtstock EUI data as there are somtimes mutiple isntacnes of buildings with the same parameters
filtered_comstock_metadata_average_eui_df = filtered_comstock_metadata_df.groupby(['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories']).mean().reset_index()
filtered_comstock_metadata_average_eui_df

#ffill the ComStock EUI data to cover buildings that dont meet exact parameters in Comstock
filtered_comstock_metadata_df.sort_values(by=['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories'], ascending = True, inplace=True)
filtered_comstock_metadata_average_eui_df = filtered_comstock_metadata_df.groupby(['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories']).mean().reset_index()
filtered_comstock_metadata_average_eui_df['Comstock EUI (BTU/SF)'] = filtered_comstock_metadata_df.groupby(['ComStock Property Type'])['Comstock EUI (BTU/SF)'].ffill()

#Map Comstock EUI to DataSet
property_comstock_merged_df = pd.merge(
    property_df,
    filtered_comstock_metadata_average_eui_df,
    on=['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories'],
    how = 'left'
    )

property_comstock_merged_df.sort_values(by=['ComStock Property Type', 'ComStock Vintage', 'ComStock Construction', 'ComStock Size Range', 'Number Of Stories'], ascending = False, inplace=True)
property_comstock_merged_df

,Unnamed: 0,Address,Property Name,PropertyType,Star Rating,Energy Star,LEED Certified,Building Class,Building Status,Rent/SF/Yr,...,Utilities,Total Area SF,Full Property Type,ComStock Construction,ComStock Vintage,ComStock Size Range,ComStock Property Type,Colorado 2026 EUI Target,Colorado 2030 EUI Target,Comstock EUI (BTU/SF)
820,820,15594 E Batavia Dr,NaN,Industrial,1,NaN,NaN,C,Existing,11.92,...,1.08,8940,Industrial: Warehouse,Wood Frame,Before 1946,5001-10000,Warehouse,42.1,32.3,27.592970
762,762,TBD E 48th Ave,Building D,Industrial,4,NaN,NaN,B,Proposed,0.00,...,1.08,338400,Industrial: Warehouse,Wood Frame,Before 1946,200001-500000,Warehouse,42.1,32.3,89.231129
765,765,TBD E 48th Ave,Building A,Industrial,4,NaN,NaN,B,Abandoned,0.00,...,1.08,1019520,Industrial: Warehouse,Wood Frame,Before 1946,200001-500000,Warehouse,42.1,32.3,89.231129
771,771,TBD 64th Ave,Building 8,Industrial,4,NaN,NaN,A,Proposed,0.00,...,1.08,1209000,Industrial: Warehouse,Wood Frame,Before 1946,200001-500000,Warehouse,42.1,32.3,89.231129
850,850,20500 E Colfax Ave,Building 6,Industrial,4,NaN,NaN,B,Deferred,0.00,...,1.08,1018456,Industrial: Warehouse,Wood Frame,Before 1946,200001-500000,Warehouse,42.1,32.3,89.231129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6465,6465,11757 E 14th Ave,Elks Lodge,Specialty,2,NaN,NaN,C,Existing,0.00,...,NaN,6222,Specialty: Lodge/Meeting Hall,Mass,1946 to 1959,5001-10000,NaN,NaN,NaN,NaN
6536,6536,1381 Iola St,Ministerios Internacional Refugio Eterno,Specialty,1,NaN,NaN,C,Existing,0.00,...,NaN,5798,Specialty: Religious Facility,Mass,1946 to 1959,5001-10000,NaN,NaN,NaN,NaN
6491,6491,1455 Boston St,NaN,Specialty,2,NaN,NaN,C,Existing,0.00,...,NaN,5000,Specialty: Religious Facility,Mass,1946 to 1959,1001-5000,NaN,NaN,NaN,NaN
6539,6539,1300 Jamaica St,First Scientist Church Of Christ,Specialty,2,NaN,NaN,C,Existing,0.00,...,NaN,4241,Specialty: Religious Facility,Mass,1946 to 1959,1001-5000,NaN,NaN,NaN,NaN


Baseline Analysis

In [71]:
#Merge Average Market Data for reference in data set
property_comstock_merged_df = pd.merge(
    property_comstock_merged_df, 
    average_market_df, 
    on = "ComStock Property Type",
    how = 'left'
    )

#Merge Average Hospitalityh market for Refence in data set
property_comstock_merged_df = pd.merge(
    property_comstock_merged_df,
    average_market_hospitality_df,
    on = "ComStock Property Type",
    how = 'left'
)

#Calculate Total Area
property_comstock_merged_df['Total Area SF'] = property_comstock_merged_df['Typical Floor Size'] * property_comstock_merged_df['Number Of Stories']


##Use best available data. If not available use average market, modelled, or other data
#Use Touchstone report EUI data if not use Comstock
property_comstock_merged_df['EUI for Analysis (BTU/SF)'] = property_comstock_merged_df['site_eui'].fillna(property_comstock_merged_df['Comstock EUI (BTU/SF)'])
    
#Use Average Market Rent if not available.
property_comstock_merged_df['Rent/SF/Yr'] = property_comstock_merged_df['Rent/SF/Yr'].replace(0, np.nan).fillna(property_comstock_merged_df['Average Market Rent/SF'])

#Fill the Baseline Rent/SF for Large Hotels with Average Revenue/Total Area
property_comstock_merged_df['Baseline Rent/SF'] = property_comstock_merged_df['Rent/SF/Yr'].fillna(property_comstock_merged_df['Average Revenue'] / property_comstock_merged_df['Total Area SF'])


#Use Average Vacancy % if Vacancy is not. Note Vancancy from costar is not a percent. Convert to percent before.
property_comstock_merged_df['Vacancy %'] = property_comstock_merged_df['Vacancy %'] / 100
property_comstock_merged_df['Vacancy Analyzed (%)'] = property_comstock_merged_df['Vacancy %'].fillna(property_comstock_merged_df['Average Vacancy Rate'])

#Calculate Yearly Rent and then Gross Rent
property_comstock_merged_df['Yearly Rent'] = property_comstock_merged_df['Baseline Rent/SF'] * property_comstock_merged_df['Total Area SF']

# Baseline Rent/Yr is either the yearly rent * vacancy unless its a Large Hotel. Large Hotels Baseline Rent is based on Yearly Revenue so vacancy is not applicable.
property_comstock_merged_df['Baseline Rent/Yr'] = property_comstock_merged_df.apply(lambda row: row['Yearly Rent'] if row['ComStock Property Type'] == 'Large Hotel' else row['Yearly Rent'] * (1 - row['Vacancy Analyzed (%)']), axis = 1)

#Remove Dollar Sign From Building Expenses
property_comstock_merged_df['Building Operating Expenses'] = property_comstock_merged_df['Building Operating Expenses'].str.replace('$', '').astype(float)
property_comstock_merged_df['Building Tax Expenses'] = property_comstock_merged_df['Building Tax Expenses'].str.replace('$', '').astype(float)

#$Calculate Total Building Expenses

#In instances where expenses are greater than 100 the expenses are not in a $/SF metric. Convert to $/SF
#Create a mask to determine where expenses are < 100
mask_opr_exp = property_comstock_merged_df['Building Operating Expenses'] < 500 
mask_tax_exp = property_comstock_merged_df['Building Tax Expenses'] < 500

# Apply the mask to create a new column 'Sum_Columns' with conditional sum
property_comstock_merged_df['Building Operating Expenses'] = property_comstock_merged_df.apply(lambda row: row['Building Operating Expenses'] if mask_opr_exp[row.name] else row['Building Operating Expenses'] / row['Total Area SF'], axis=1)
property_comstock_merged_df['Building Tax Expenses'] = property_comstock_merged_df.apply(lambda row: row['Building Tax Expenses'] if mask_tax_exp[row.name] else row['Building Tax Expenses'] / row['Total Area SF'], axis=1)

#In instances where building expenses and taxes expenses are the same the only expenses is tax. If different sum together. If not use only one of the expenses
# Create a mask to identify rows where Building Expenses and Tax Expenses are the same
mask = property_comstock_merged_df['Building Operating Expenses'] == property_comstock_merged_df['Building Tax Expenses']

# Apply the mask to create a new column 'Sum_Columns' with conditional sum
property_comstock_merged_df['Building Total Operating Expenses/SF'] = property_comstock_merged_df.apply(lambda row: row['Building Operating Expenses'] if mask[row.name] else row['Building Operating Expenses'] + row['Building Tax Expenses'], axis=1)
property_comstock_merged_df['Building Total Operating Expenses'] = property_comstock_merged_df['Building Total Operating Expenses/SF'] * property_comstock_merged_df['Total Area SF']

#
property_comstock_merged_df['Average Building Expenses Based on SF'] = property_comstock_merged_df['Building Total Operating Expenses/SF'].mean() * property_comstock_merged_df['Total Area SF']
property_comstock_merged_df['Baseline Total Operating Expenses'] = property_comstock_merged_df['Building Total Operating Expenses'].replace(0, np.nan).fillna(property_comstock_merged_df['Average Building Expenses Based on SF'])


#Calculate Net Operating Income
property_comstock_merged_df['Baseline Net Operating Income'] = property_comstock_merged_df['Baseline Rent/Yr'] - property_comstock_merged_df['Baseline Total Operating Expenses']

#Use the Property Price if not availbe use Median Sale Price/SF 
property_comstock_merged_df['Calculated Property Price'] = property_comstock_merged_df.apply(lambda row:  row['Average Market Sale Price/Room'] * row['Rooms'] if row['ComStock Property Type'] == 'Large Hotel' else row['Median Price/Bldg SF'] * row['Total Area SF'], axis = 1)
property_comstock_merged_df['Baseline Property Price'] = property_comstock_merged_df['Last Sale Price'].replace(0, np.nan).fillna(property_comstock_merged_df['Calculated Property Price'])

#Use Cap Rate Available in CoStar if not Calculate the Cap Rate. Note CoStar Cap Rate is not in Percent format, convert before utilizing
property_comstock_merged_df['Cap Rate'] = property_comstock_merged_df['Cap Rate'] / 100
property_comstock_merged_df['Calculated Cap Rate'] = property_comstock_merged_df['Baseline Net Operating Income'] / property_comstock_merged_df['Baseline Property Price']
property_comstock_merged_df['Baseline Cap Rate'] = property_comstock_merged_df['Calculated Cap Rate']

# property_comstock_merged_df['Baseline Cap Rate'] = property_comstock_merged_df['Cap Rate'].replace(0, np.nan).fillna(property_comstock_merged_df['Calculated Cap Rate'])

#Determine if Building Meets Colorado BPS Requirements
property_comstock_merged_df['Greater Than 50k SF'] = property_comstock_merged_df['Total Area SF'] > 49999
property_comstock_merged_df['Greater Than 2026 EUI'] = property_comstock_merged_df['EUI for Analysis (BTU/SF)'] > property_comstock_merged_df['Colorado 2026 EUI Target']
property_comstock_merged_df['Greater Than 2030 EUI'] = property_comstock_merged_df['EUI for Analysis (BTU/SF)'] > property_comstock_merged_df['Colorado 2030 EUI Target']


property_comstock_merged_df

,Unnamed: 0,Address,Property Name,PropertyType,Star Rating,Energy Star,LEED Certified,Building Class,Building Status,Rent/SF/Yr,...,Average Building Expenses Based on SF,Baseline Total Operating Expenses,Baseline Net Operating Income,Calculated Property Price,Baseline Property Price,Calculated Cap Rate,Baseline Cap Rate,Greater Than 50k SF,Greater Than 2026 EUI,Greater Than 2030 EUI
0,820,15594 E Batavia Dr,NaN,Industrial,1,NaN,NaN,C,Existing,11.92,...,7.389033e+04,2.655180e+04,7.327728e+04,1788000.0,1788000.0,0.040983,0.040983,False,False,False
1,762,TBD E 48th Ave,Building D,Industrial,4,NaN,NaN,B,Proposed,11.84,...,2.796922e+06,2.796922e+06,9.564821e+05,67680000.0,67680000.0,0.014132,0.014132,True,True,True
2,765,TBD E 48th Ave,Building A,Industrial,4,NaN,NaN,B,Abandoned,11.84,...,8.426473e+06,8.426473e+06,2.881657e+06,203904000.0,203904000.0,0.014132,0.014132,True,True,True
3,771,TBD 64th Ave,Building 8,Industrial,4,NaN,NaN,A,Proposed,11.84,...,9.992551e+06,9.992551e+06,3.417219e+06,241800000.0,241800000.0,0.014132,0.014132,True,True,True
4,850,20500 E Colfax Ave,Building 6,Industrial,4,NaN,NaN,B,Deferred,11.84,...,8.417678e+06,1.099932e+06,1.019640e+07,203691200.0,203691200.0,0.050058,0.050058,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6609,6465,11757 E 14th Ave,Elks Lodge,Specialty,2,NaN,NaN,C,Existing,NaN,...,5.142568e+04,1.679940e+04,NaN,NaN,800000.0,NaN,NaN,False,False,False
6610,6536,1381 Iola St,Ministerios Internacional Refugio Eterno,Specialty,1,NaN,NaN,C,Existing,NaN,...,4.792127e+04,4.792127e+04,NaN,NaN,300000.0,NaN,NaN,False,False,False
6611,6491,1455 Boston St,NaN,Specialty,2,NaN,NaN,C,Existing,NaN,...,4.132569e+04,4.132569e+04,NaN,NaN,550000.0,NaN,NaN,False,False,False
6612,6539,1300 Jamaica St,First Scientist Church Of Christ,Specialty,2,NaN,NaN,C,Existing,NaN,...,3.505245e+04,9.287790e+03,NaN,NaN,305000.0,NaN,NaN,False,False,False


Calculate the BPS Fines

In [72]:
#Set the Colorado BPS Fine assumes 10 year average based on current Colorado Rules
co_benchmarking_fee_yr = (100+5000+2000*9)/10
co_bps_fee_yr = (24000+60000*9)/10
co_tot_bps_fee_yr = co_benchmarking_fee_yr + co_bps_fee_yr

#Set the $/SF Fine. Fine will be a flat $/SF if the building is over the EUI target
per_sf_fee_yr = 2

#Set the $/kBTU Fine. Fine will be a flat $/kBTU based on the difference of the actual EUI to the Target EUI
per_kbtu_fee_yr = .05


#Calculate the Fees and remove non applicable baseline values to use in analysis
property_comstock_merged_df['Baseline Applicable Cap Rate'] = property_comstock_merged_df.apply(lambda row: row['Baseline Cap Rate'] if (row['Greater Than 50k SF'] and row['Greater Than 2030 EUI']) else np.nan, axis = 1)
property_comstock_merged_df['Baseline Applicable Rent/SF'] = property_comstock_merged_df.apply(lambda row: row['Baseline Rent/SF'] if (row['Greater Than 50k SF'] and row['Greater Than 2030 EUI']) else np.nan, axis = 1)
property_comstock_merged_df['Colorado BPS Fine'] = property_comstock_merged_df.apply(lambda row: co_tot_bps_fee_yr if (row['Greater Than 50k SF'] and row['Greater Than 2030 EUI']) else np.nan, axis = 1)
property_comstock_merged_df['$/SF BPS Fine'] = property_comstock_merged_df.apply(lambda row: per_sf_fee_yr*row['Total Area SF'] if (row['Greater Than 50k SF'] and row['Greater Than 2030 EUI']) else np.nan, axis = 1)
property_comstock_merged_df['$/kBTU BPS Fine'] = property_comstock_merged_df.apply(lambda row: per_kbtu_fee_yr*((row['EUI for Analysis (BTU/SF)'] - row['Colorado 2030 EUI Target'])*row['Total Area SF']) if (row['Greater Than 50k SF'] and row['Greater Than 2030 EUI']) else np.nan, axis = 1)

property_comstock_merged_df

,Unnamed: 0,Address,Property Name,PropertyType,Star Rating,Energy Star,LEED Certified,Building Class,Building Status,Rent/SF/Yr,...,Calculated Cap Rate,Baseline Cap Rate,Greater Than 50k SF,Greater Than 2026 EUI,Greater Than 2030 EUI,Baseline Applicable Cap Rate,Baseline Applicable Rent/SF,Colorado BPS Fine,$/SF BPS Fine,$/kBTU BPS Fine
0,820,15594 E Batavia Dr,NaN,Industrial,1,NaN,NaN,C,Existing,11.92,...,0.040983,0.040983,False,False,False,NaN,NaN,NaN,NaN,NaN
1,762,TBD E 48th Ave,Building D,Industrial,4,NaN,NaN,B,Proposed,11.84,...,0.014132,0.014132,True,True,True,0.014132,11.84,58710.0,676800.0,9.632747e+05
2,765,TBD E 48th Ave,Building A,Industrial,4,NaN,NaN,B,Abandoned,11.84,...,0.014132,0.014132,True,True,True,0.014132,11.84,58710.0,2039040.0,2.902121e+06
3,771,TBD 64th Ave,Building 8,Industrial,4,NaN,NaN,A,Proposed,11.84,...,0.014132,0.014132,True,True,True,0.014132,11.84,58710.0,2418000.0,3.441487e+06
4,850,20500 E Colfax Ave,Building 6,Industrial,4,NaN,NaN,B,Deferred,11.84,...,0.050058,0.050058,True,True,True,0.050058,11.84,58710.0,2036912.0,2.899093e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6609,6465,11757 E 14th Ave,Elks Lodge,Specialty,2,NaN,NaN,C,Existing,NaN,...,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN
6610,6536,1381 Iola St,Ministerios Internacional Refugio Eterno,Specialty,1,NaN,NaN,C,Existing,NaN,...,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN
6611,6491,1455 Boston St,NaN,Specialty,2,NaN,NaN,C,Existing,NaN,...,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN
6612,6539,1300 Jamaica St,First Scientist Church Of Christ,Specialty,2,NaN,NaN,C,Existing,NaN,...,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN


Perform the Economic Analysis of the Fines at a Property Level

In [73]:
bps_fine_list = ['Colorado', '$/SF', '$/kBTU']

for fine in bps_fine_list:
    property_comstock_merged_df[f'{fine} BPS % Increase in Expenses'] = ((property_comstock_merged_df[f'{fine} BPS Fine'] + property_comstock_merged_df['Baseline Total Operating Expenses'])/property_comstock_merged_df['Baseline Total Operating Expenses']) - 1
    property_comstock_merged_df[f'{fine} BPS % Increase in Rent/SF'] = (((property_comstock_merged_df[f'{fine} BPS Fine']/property_comstock_merged_df['Total Area SF']) + property_comstock_merged_df['Baseline Rent/SF'])/property_comstock_merged_df['Baseline Rent/SF']) - 1
    property_comstock_merged_df[f'{fine} BPS New Rent/SF'] = ((property_comstock_merged_df[f'{fine} BPS Fine']/property_comstock_merged_df['Total Area SF']) + property_comstock_merged_df['Baseline Rent/SF'])
    property_comstock_merged_df[f'{fine} BPS New Cap Rate'] = ((property_comstock_merged_df['Baseline Net Operating Income'] - property_comstock_merged_df[f'{fine} BPS Fine'])/property_comstock_merged_df['Baseline Property Price'])
    property_comstock_merged_df[f'{fine} BPS Cap Rate Change'] = property_comstock_merged_df[f'{fine} BPS New Cap Rate'] - property_comstock_merged_df['Baseline Cap Rate']


property_comstock_merged_df.to_csv('BPS_Economic_Property_Analysis.csv')

Map the Property Level Analysis to the Tenant Level to Analyze Impact to Tenants

In [74]:
analysis_column_headers = ['Address','Baseline Applicable Rent/SF', 'Baseline Applicable Cap Rate']
for fine in bps_fine_list:
    analysis_column_headers.append(f'{fine} BPS % Increase in Expenses')
    analysis_column_headers.append(f'{fine} BPS % Increase in Rent/SF')
    analysis_column_headers.append(f'{fine} BPS New Rent/SF')
    analysis_column_headers.append(f'{fine} BPS New Cap Rate')
    analysis_column_headers.append(f'{fine} BPS Cap Rate Change')

#add in the Identified as disadvantaged column for later analysis
analysis_column_headers.append('Identified as disadvantaged')
analysis_column_headers.append('ComStock Property Type')

#merged Property data to tenant level data
property_analysis_df = property_comstock_merged_df[analysis_column_headers]
dropped_duplicates_property_analysis_df = property_analysis_df.drop_duplicates(subset = 'Address', keep = 'last')
tenant_analysis_df = pd.merge(tenant_df,dropped_duplicates_property_analysis_df, on = 'Address', how = 'left')

tenant_analysis_df.to_csv('BPS_Economic_Tenant_Analysis.csv')

